# Data Types

Sample code to illustrate how .NET data types and associated metadata are represented
in OpenAPI documents generated by ASP.NET.

C# classes or records used in request or response bodies are represented as schemas
in the generated OpenAPI document.
By default, only public properties are represented in the schema, but there are [JsonSerializerOptions]
to also create schema properties for fields.

When the System.Text.Json [PropertyNamingPolicy] is set to camel-case (this is the default
in ASP.NET web applications), property names in a schema are the camel-case form
of the class or record property name.
The [JsonPropertyNameAttribute] can be used on an individual property to specify the name
of the property in the schema.

[JsonPropertyNameAttribute]: https://docs.microsoft.com/dotnet/api/system.text.json.serialization.jsonpropertynameattribute
[JsonSerializerOptions]: https://docs.microsoft.com/dotnet/api/system.text.json.jsonserializeroptions
[PropertyNamingPolicy]: https://docs.microsoft.com/dotnet/api/system.text.json.jsonserializeroptions.propertynamingpolicy

## type and format

The JSON Schema library maps standard C# types to OpenAPI `type` and `format` as follows:

| C# Type        | OpenAPI `type` | OpenAPI `format` |
| -------------- | -------------- | ---------------- |
| int            | integer        | int32            |
| long           | integer        | int64            |
| short          | integer        | int16            |
| byte           | integer        | uint8            |
| float          | number         | float            |
| double         | number         | double           |
| decimal        | number         | double           |
| bool           | boolean        |                  |
| string         | string         |                  |
| char           | string         | char             |
| byte[]         | string         | byte             |
| DateTimeOffset | string         | date-time        |
| DateOnly       | string         | date             |
| TimeOnly       | string         | time             |
| Uri            | string         | uri              |
| Guid           | string         | uuid             |
| object         | _omitted_      |                  |
| dynamic        | _omitted_      |                  |

Note that object and dynamic types have _no_ type defined in the OpenAPI because
these can contain data of any type.

The `type` and `format` can also be set with a [Schema Transformer]. For example,
you may want the `format` of decimal types to be `decimal` instead of `double`.

In [6]:
jq 'include "openapi"; show_schemas(.DataTypes)' DataTypes/DataTypes.json

{
  "schema": {
    "DataTypes": {
      "type": "object",
      "properties": {
        "int": {
          "type": "integer",
          "format": "int32"
        },
        "long": {
          "type": "integer",
          "format": "int64"
        },
        "short": {
          "type": "integer",
          "format": "int16"
        },
        "byte": {
          "type": "integer",
          "format": "uint8"
        },
        "float": {
          "type": "number",
          "format": "float"
        },
        "double": {
          "type": "number",
          "format": "double"
        },
        "decimal": {
          "type": "number",
          "format": "double"
        },
        "bool": {
          "type": "boolean"
        },
        "string": {
          "type": "string"
        },
        "char": {
          "maxLength": 1,
          "minLength": 1,
          "type": "string",
          "format": "char"
        },
        "byteArray": {
          "type": "string",
          

In [ ]:
curl -s -D - -X POST `
  -H "content-type: application/json" `
  -d '{
        "int": 123,
        "long": 123456789012345,
        "short": 12345,
        "float": 123.45,
        "double": 123.456789,
        "bool": true,
        "string": "example",
        "char": "A",
        "byte": 255,
        "byteArray": "SGVsbG8gV29ybGQ=",
        "decimal": 12345.6789,
        "object": "can be any type",
        "dynamic": 42,
        "dateTimeOffset": "2023-10-01T12:34:56.789Z",
        "dateOnly": "2023-10-01",
        "timeOnly": "12:34:56",
        "uri": "http://example.com",
        "uuid": "123e4567-e89b-12d3-a456-426614174000"
  }' `
  http://localhost:5187/data-types

HTTP/1.1 200 OK
Content-Type: application/json; charset=utf-8
Date: Thu, 10 Oct 2024 21:01:25 GMT
Server: Kestrel
Transfer-Encoding: chunked

{
  "int": 123,
  "long": 123456789012345,
  "short": 12345,
  "byte": 255,
  "float": 123.45,
  "double": 123.456789,
  "decimal": 12345.6789,
  "bool": true,
  "string": "example",
  "char": "A",
  "byteArray": "SGVsbG8gV29ybGQ=",
  "dateTimeOffset": "2023-10-01T12:34:56.789+00:00",
  "dateOnly": "2023-10-01",
  "timeOnly": "12:34:56",
  "uri": "http://example.com",
  "uuid": "123e4567-e89b-12d3-a456-426614174000",
  "object": "can be any type",
  "dynamic": 42
}


## Using attributes to add metadata

ASP.NET will use metadata from attributes on class or record properties to set metadata
on the corresponding properites of the generated schema.
The following table summarizes attributes from the System.ComponentModel namespace
that provide metadata for the generated schema.

| Attribute                    | Description |
| ---------------------------- | ----------- |
| [DescriptionAttribute]       | Sets the `description` of a property in the schema. |
| [RequiredAttribute]          | Marks a property as `required` in the schema. |
| [DefaultValueAttribute]      | Sets the `default` value of a property in the schema. |
| [RangeAttribute]             | Sets the `minimum` and `maximum` value of an integer or number. |
| [MinLengthAttribute]         | Sets the `minLength` of a string. |
| [MaxLengthAttribute]         | Sets the `maxLength` of a string. |
| [RegularExpressionAttribute] | Sets the `pattern` of a string. |

Note that in controller-based apps, these attributes add filters to the operation
to validate any incoming data satifies the constraints.
In Minimal APIs, these attributes will set the metadata in the generated schema
but validation must be performed by the route handler.

[DefaultValueAttribute]: https://learn.microsoft.com/dotnet/api/system.componentmodel.defaultvalueattribute
[DescriptionAttribute]: https://learn.microsoft.com/dotnet/api/system.componentmodel.descriptionattribute
[MinLengthAttribute]: https://learn.microsoft.com/dotnet/api/system.componentmodel.dataannotations.minlengthattribute
[MaxLengthAttribute]: https://learn.microsoft.com/dotnet/api/system.componentmodel.dataannotations.maxlengthattribute
[RangeAttribute]: https://learn.microsoft.com/dotnet/api/system.componentmodel.dataannotations.rangeattribute
[RegularExpressionAttribute]: https://learn.microsoft.com/dotnet/api/system.componentmodel.dataannotations.regularexpressionattribute
[RequiredAttribute]: https://learn.microsoft.com/dotnet/api/system.componentmodel.dataannotations.requiredattribute

In [35]:
jq 'include "openapi"; show_schemas(.Metadata)' DataTypes/DataTypes.json

{
  "schema": {
    "Metadata": {
      "required": [
        "requiredAttribute"
      ],
      "type": "object",
      "properties": {
        "description": {
          "type": "string",
          "description": "A description of the property"
        },
        "requiredAttribute": {
          "type": "integer",
          "format": "int32"
        },
        "defaultValueAttribute": {
          "type": "integer",
          "format": "int32",
          "default": 42
        },
        "intWithRange": {
          "maximum": 100,
          "minimum": 1,
          "type": "integer",
          "format": "int32"
        },
        "doubleWithRange": {
          "maximum": 1,
          "minimum": 0,
          "type": "number",
          "format": "double"
        },
        "stringWithMaxLength": {
          "maxLength": 63,
          "type": "string"
        },
        "stringWithMinLength": {
          "minLength": 1,
          "type": "string"
        },
        "stringWithPattern": {


## Other sources of metadata for generated schemas

### required

Properties can also be marked as `required` with the [required] modifier.

[required]: https://learn.microsoft.com/dotnet/csharp/language-reference/proposals/csharp-11.0/required-members#required-modifier

### nullable

Properties defined as a nullable value or reference type will have `nullable: true` in the generated schema.
This is consistent with the default behavior of the System.Text.Json deserializer, which accepts `null` as a
valid value for a nullable property.

### additionalProperties

Schemas are generated without an `additionalProperties` assertion by default, which implies the default of `true`.
This is consistent with the default behavior of the System.Text.Json deserializer, which silently ignores
additional properties in a JSON object.

If the additional properties of a schema should only have values of a specific type,
define the property or class as a Dictionary<string, type>.
The key type for the dictionary must be `string`.
This will generate a schema with `additionalProperties` specifying the schema for "type" as the required value types.

In [38]:
jq 'include "openapi"; show_schemas(.MoreMetadata)' DataTypes/DataTypes.json

{
  "schema": {
    "MoreMetadata": {
      "required": [
        "requiredModifier"
      ],
      "type": "object",
      "properties": {
        "requiredModifier": {
          "type": "integer",
          "format": "int32"
        },
        "nonNullableRef": {
          "type": "string"
        },
        "nonNullableValue": {
          "type": "integer",
          "format": "int32"
        },
        "nullableRef": {
          "type": "string",
          "nullable": true
        },
        "nullableValue": {
          "type": "integer",
          "format": "int32",
          "nullable": true
        },
        "dictionary": {
          "type": "object",
          "additionalProperties": {
            "type": "string"
          }
        }
      }
    }
  }
}



### enum

Enum types in C# are integer-based, but can be represented as strings in JSON with a [JsonConverterAttribute] and a [JsonStringEnumConverter]. When an enum type is represented as a string in JSON, the schema will have an `enum` property with the string values of the enum.

An enum type without a [JsonConverterAttribute] will be defined as `type: integer` in the generated schema.

Note: the [AllowedValuesAttribute] does not set the `enum` values of a property.

[JsonConverterAttribute]: https://learn.microsoft.com/dotnet/api/system.text.json.serialization.jsonconverterattribute
[JsonStringEnumConverter]: https://learn.microsoft.com/dotnet/api/system.text.json.serialization.jsonstringenumconverter
[AllowedValuesAttribute]: https://learn.microsoft.com/dotnet/api/system.componentmodel.dataannotations.allowedvaluesattribute

In [39]:
jq -r 'include "openapi"; show_schemas(.Enums)' DataTypes/DataTypes.json

{
  "schema": {
    "Enums": {
      "type": "object",
      "properties": {
        "enumAsString": {
          "enum": [
            "Sunday",
            "Monday",
            "Tuesday",
            "Wednesday",
            "Thursday",
            "Friday",
            "Saturday"
          ]
        },
        "enum": {
          "type": "integer"
        },
        "allowedValues": {
          "type": "string"
        }
      }
    }
  }
}



### Metadata for polymorphic types

Use the System.Text.Json [JsonPolymorphicAttribute] and [JsonDerivedTypeAttribute] attributes on a parent class to
to specify the discriminator field and subtypes for a polymorphic type.

The [JsonDerivedTypeAttribute] adds the discriminator field to the schema for each subclass,
with an enum specifying the specific discriminator value for the subclass.
This attribute also modifies the constructor of each derived class to set the discriminator value.

An abstract class with a [JsonPolymorphicAttribute] attribute will have a `discriminator` field in the schema.

[JsonDerivedTypeAttribute]: https://learn.microsoft.com/dotnet/api/system.text.json.serialization.jsonderivedtypeattribute
[JsonPolymorphicAttribute]: https://learn.microsoft.com/dotnet/api/system.text.json.serialization.jsonpolymorphicattribute

In [37]:
jq -r 'include "openapi"; show_schemas(.Shape,.ShapeCircle,.ShapeSquare,.ShapeTriangle)' DataTypes/DataTypes.json

{
  "schema": {
    "Shape": {
      "required": [
        "shapeType"
      ],
      "type": "object",
      "anyOf": [
        {
          "$ref": "#/components/schemas/ShapeCircle"
        },
        {
          "$ref": "#/components/schemas/ShapeSquare"
        },
        {
          "$ref": "#/components/schemas/ShapeTriangle"
        }
      ],
      "discriminator": {
        "propertyName": "shapeType",
        "mapping": {
          "circle": "#/components/schemas/ShapeCircle",
          "square": "#/components/schemas/ShapeSquare",
          "triangle": "#/components/schemas/ShapeTriangle"
        }
      }
    },
    "ShapeCircle": {
      "properties": {
        "shapeType": {
          "enum": [
            "circle"
          ],
          "type": "string"
        },
        "radius": {
          "type": "number",
          "format": "double"
        },
        "color": {
          "type": "string"
        },
        "sides": {
          "type": "integer",
          "format


A concrete class with a [JsonPolymorphicAttribute] attribute will not have a `discriminator` field.
OpenAPI requires that the discriminator property be a required property in the schema,
but since the discriminator property is not defined in the concrete base class, the schema cannot include a `discriminator` field.

[JsonDerivedTypeAttribute]: https://learn.microsoft.com/dotnet/api/system.text.json.serialization.jsonderivedtypeattribute
[JsonPolymorphicAttribute]: https://learn.microsoft.com/dotnet/api/system.text.json.serialization.jsonpolymorphicattribute


In [37]:
jq -r 'include "openapi"; show_schemas(.Pet,.PetDog,.PetCat,.PetFish)' DataTypes/DataTypes.json

{
  "schema": {
    "Pet": {
      "type": "object",
      "anyOf": [
        {
          "$ref": "#/components/schemas/PetDog"
        },
        {
          "$ref": "#/components/schemas/PetCat"
        },
        {
          "$ref": "#/components/schemas/PetFish"
        },
        {
          "$ref": "#/components/schemas/PetBase"
        }
      ]
    },
    "PetDog": {
      "required": [
        "petType"
      ],
      "properties": {
        "petType": {
          "enum": [
            "dog"
          ],
          "type": "string"
        },
        "breed": {
          "type": "string"
        },
        "name": {
          "type": "string"
        },
        "age": {
          "type": "integer",
          "format": "int32"
        }
      }
    },
    "PetCat": {
      "required": [
        "petType"
      ],
      "properties": {
        "petType": {
          "enum": [
            "cat"
          ],
          "type": "string"
        },
        "declawed": {
          "typ